In [2]:
pip install tflearn 

     |████████████████████████████████| 98 kB 2.5 MB/s eta 0:00:01
  Created wheel for tflearn: filename=tflearn-0.3.2-py3-none-any.whl size=128208 sha256=e619da87bf195d689f52613d083b9bafac2f6b3851489f42b1f587fa636507fb
  Stored in directory: /home/jovyan/.cache/pip/wheels/8e/b7/55/b387552a28a2fefa559a362ffe17c97fac7e618ef948930b3c
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install nltk

     |████████████████████████████████| 1.4 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 660 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 7.1 MB/s  eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434675 sha256=8928c14bac1aad2cc9cf1e0e1d8cfa66909e44216707ced4505ee80e41521d6b
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer 
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import tensorflow as tf
import numpy as np
import tflearn 
import random
import json 

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
with open('intents.json') as json_data:
    intents=json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes=sorted(list(set(classes)))

In [7]:
training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
tf.reset_default_graph()
#build
net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')  #neural network classifier tasks 

Instructions for updating:
Use tf.cast instead.


In [10]:
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.87292 | time: 0.071s
| Adam | epoch: 1000 | loss: 0.87292 - acc: 0.9535 -- iter: 24/31
Training Step: 4000  | total loss: 0.78615 | time: 0.073s
| Adam | epoch: 1000 | loss: 0.78615 - acc: 0.9581 -- iter: 31/31
--
INFO:tensorflow:/home/jovyan/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [13]:
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [14]:
with open('intents.json') as json_data:
    intents=json.load(json_data)

In [15]:
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/jovyan/model.tflearn


In [16]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [17]:
def bow(sentence,words,show_details=False):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print('found in bag : %s' %w)
    return (np.array(bag))


In [32]:
context={}
ERROR_THRESHOLD=0.30
def classify(sentence):
    results=model.predict([bow(sentence,words)])[0]
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    result_list=[]
    for r in results:
        result_list.append((classes[r[0]],r[1]))
    return result_list

In [35]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [36]:
classify('What are your hours of operation?')

[('hours', 0.99865484)]

In [37]:
response('What are your hours of operation?')

We're open every day 9am-9pm


In [38]:
response('What is menu for today')

Today's special is Chicken Tikka


In [39]:
response('do you accept credit card')

We accept VISA, Mastercard and AMEX


In [40]:
response('where is my order')

Today's special is Chicken Tikka


In [41]:
response('cancel my order')

Good to see you again


In [42]:
response('what is location of restaurant')

We are on the intersection of London Alley and Bridge Avenue.


In [43]:
response('bye')

Bye! Come back again soon.


In [46]:
response('food')

Yes, we provide home delivery through UBER Eats and Zomato?
